In [1]:
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import re

In [ ]:
# taking the search input from the user
base_url = 'https://www.linkedin.com/jobs/search/?currentJobId=3568976147&geoId=103644278&keywords='

searchstring = input('enter the search string =>> ')
searchlist = searchstring.split()
i = 0

# adding the search input to the search string
while i < (len(searchlist)) :
  base_url = base_url + searchlist[i]
  if i == (len(searchlist) - 1):
    break
  base_url = base_url + '%20'
  i = i + 1

#final url before adding page number  
base_url = base_url + '&location=United%20States&refresh=true'

print('Search string generated ==> ' , base_url)

enter the search string =>> System Analyst
Search string generated ==>  https://www.linkedin.com/jobs/search/?currentJobId=3568976147&geoId=103644278&keywords=System%20Analyst&location=United%20States&refresh=true


In [ ]:
#initializing the lists to store values
job_titles = []
job_companies = []
job_locations = []
job_links = []
job_postingtimes=[]

i = 0
total = 0

#taking user input about the number of pages
pagecounter = input('how many pages are needed =>> ')
pagecounter = int(pagecounter)

#iterating through pages
while i < pagecounter:
  print('page number =>',i)
  if i == 0:
    url = base_url
  else:
    start = i*25
    startstring = str(start)
    url = base_url + '&start=' + startstring
  req = Request(url , headers={'User-Agent': 'Chrome/70.0.3538.77'})
  page = urlopen(req).read()
  soup = BeautifulSoup(page, "html.parser")
  jobs = soup.find_all('div', class_='base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card')

  for job in jobs:
    job_title = job.find('h3', class_='base-search-card__title').text.strip()
    job_titles.append(job_title)
    job_company = job.find('h4', class_='base-search-card__subtitle').text.strip()
    job_companies.append(job_company)
    job_location = job.find('span', class_='job-search-card__location').text.strip()
    job_locations.append(job_location)
    job_link = job.find('a', class_='base-card__full-link')['href']
    job_links.append(job_link)
    
    try:
      job_postingtime=job.find('time', class_=re.compile('^job-search-card__listdate')).text.strip()
    except:
      job_postingtime='N/A'
    job_postingtimes.append(job_postingtime)
    
  i = i + 1
#adding a delay between scraping
  if i < 30:
    buffer = 15
  elif i < 65:
    buffer = 25
  else:
    buffer = 60
  #wait = buffer + 1.02**i + random.randint(1,15) + random.randint(1,5)
  wait = buffer + random.randint(1,5)
  print('paused for ',round(wait,1),' seconds')
  total = total + wait
  time.sleep(wait)

#Summary of the scrap completed  
print('------------------------------')
print('total entries retrieved',len(job_titles))
print('time taken ',round((total/60),1),' minutes')

how many pages are needed =>> 10
page number => 0
paused for  20  seconds
page number => 1
paused for  19  seconds
page number => 2
paused for  18  seconds
page number => 3
paused for  17  seconds
page number => 4
paused for  16  seconds
page number => 5
paused for  17  seconds
page number => 6
paused for  18  seconds
page number => 7
paused for  18  seconds
page number => 8
paused for  17  seconds
page number => 9
paused for  18  seconds
------------------------------
total entries retrieved 250
time taken  3.0  minutes


In [ ]:
job_levels=[]
job_types=[]
job_functions=[]
job_industries=[]
count=0
for job in job_links:
  if count%5==0:
    time.sleep(5)
  newurl=job
  newreq=Request(newurl , headers={'User-Agent': 'Chrome/70.0.3538.77'})
  newpage = urlopen(newreq).read()
  newsoup = BeautifulSoup(newpage, "html.parser")
    
  job_detail=newsoup.find_all('span',class_='description__job-criteria-text description__job-criteria-text--criteria')
  try:
    job_level=job_detail[0].text.strip()
  except:
    job_level='N/A'
  job_levels.append(job_level)
  
  try:
    job_type=job_detail[1].text.strip()
  except:
     job_type='N/A'
  job_types.append(job_type)

  try:
    job_function=job_detail[2].text.strip()
  except:
    job_function='N/A'
  job_functions.append(job_function)

  try:
    job_industry=job_detail[3].text.strip()
  except:
    job_industry='N/A'
  job_industries.append(job_industry)

  count=count+1

In [ ]:
# creating a data frame and storing values there:
test_df = pd.DataFrame({'Job title': job_titles,
                       'Company name': job_companies,
                       'Location': job_locations,
                       'Posted on':job_postingtimes,
                       'Job level ':job_levels,
                       'Job Type' : job_types,
                       'Job Function': job_functions,
                       'Industry ' :job_industries
})
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job title     250 non-null    object
 1   Company name  250 non-null    object
 2   Location      250 non-null    object
 3   Posted on     250 non-null    object
 4   Job level     250 non-null    object
 5   Job Type      250 non-null    object
 6   Job Function  250 non-null    object
 7   Industry      250 non-null    object
dtypes: object(8)
memory usage: 15.8+ KB
None


,Job title,Company name,Location,Posted on,Job level,Job Type,Job Function,Industry
0,System Analyst,Meriplex,"Houston, TX",6 days ago,Mid-Senior level,Full-time,Information Technology,Internet Publishing
1,Systems Analyst - Early Career,Lockheed Martin,"Huntsville, AL",1 week ago,Entry level,Full-time,Information Technology,Defense and Space Manufacturing
2,systems analyst I - ST,Starbucks,"Seattle, WA",6 days ago,Not Applicable,Full-time,Information Technology and Engineering,"Food and Beverage Services, Manufacturing, and..."
3,Systems Analyst,Momento USA LLC,"Austin, TX",2 weeks ago,Entry level,Full-time,Information Technology,IT Services and IT Consulting
4,System Engineer,MedReview Inc.,"New York, NY",3 weeks ago,Mid-Senior level,Full-time,Information Technology,Internet Publishing
...,...,...,...,...,...,...,...,...
245,Computer System Analyst - Aviation SASMO,The Oryza Group,"Lexington, KY",1 week ago,Entry level,Full-time,Information Technology,Information Technology & Services
246,Systems Analyst I,Elevance Health,"Miles, TX",3 weeks ago,Entry level,Full-time,Information Technology,Hospitals and Health Care
247,Systems Analyst I,Elevance Health,"New Jersey, United States",3 weeks ago,Entry level,Full-time,Information Technology,Hospitals and Health Care
248,Lead System Analyst,Franklin Templeton,"Stamford, CT",1 week ago,Mid-Senior level,Full-time,Information Technology,Financial Services and Investment Management


In [ ]:
test_df.to_csv('SystemAnalyst.csv', index=False)